In [22]:
import pandas as pd


df = pd.read_csv('fraud detection.csv')

In [23]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [24]:
df.shape

(6362620, 11)

<h1> HANDLING INCORRECT VALUES </H1>



<p>Below is to check if there are zero balance on the original old and new balance.<p>



In [4]:
df[(df['type']=='CASH_IN') & (df['oldbalanceOrg']==0) & (df['newbalanceOrig']==0) & (df['amount']!=0)]


,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud


Turns out in transaction type of Cash-In there were no errors/missing values since, after cash-in or top up there were no zero newbalanceOrig

In [5]:
mask1 = df.loc[(df['oldbalanceOrg']==0)&((df['newbalanceDest'] - df['amount']) == df['oldbalanceDest'])&(df['amount']!=0),'amount'].values

# Copyng values of amount feature

## Data before the fixing

In [25]:
df.iloc[[852]]

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
852,1,CASH_OUT,6039.61,C1893563925,0.0,0.0,C1653844940,31164.0,37203.61,0,0


Checking if the newbalanceDest amount - amount of transaction equals to the amount of oldbalanceDest it means the money is add up correctly. <br>
And if the money add up correctly but the oldbalanceOrg is zero means it is a missing value

In [6]:

df.loc[((df['oldbalanceOrg']==0)&((df['newbalanceDest'] - df['amount']) == df['oldbalanceDest'])&(df['amount']!=0)),'oldbalanceOrg'] = mask1

# Paste the values of amount into oldbalanceOrg

In [7]:
# check to see if its filled

df.iloc[[852]]


,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
852,1,CASH_OUT,6039.61,C1893563925,6039.61,0.0,C1653844940,31164.0,37203.61,0,0


The old balance succesfully change from zero into the amount of transaction.

In [8]:
# Since the data is imbalance, I will Drop zero values oldbalanceOrg which isFraud = 0.

df.drop(df.loc[(df['oldbalanceOrg']==0) & (df['newbalanceOrig']==0) & (df['amount']!=0) & (df['isFraud']==0)].index, inplace=True)

In [9]:
# Checking if its dropped or not

df.loc[(df['oldbalanceOrg']==0) & (df['newbalanceOrig']==0) & (df['amount']!=0) & (df['isFraud']==0)]


,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud


In [10]:
df.iloc[[852]]

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
1240,1,DEBIT,1280.14,C626393072,15159.0,13878.86,C1112414583,11898.0,67762.92,0,0


In [11]:
df.shape

(4863726, 11)

In [12]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [13]:
drop = df.drop(['nameOrig', 'nameDest', 'isFlaggedFraud'], axis = 1)

In [14]:
drop.head()

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,1,PAYMENT,9839.64,170136.0,160296.36,0.0,0.0,0
1,1,PAYMENT,1864.28,21249.0,19384.72,0.0,0.0,0
2,1,TRANSFER,181.00,181.0,0.00,0.0,0.0,1
3,1,CASH_OUT,181.00,181.0,0.00,21182.0,0.0,1
4,1,PAYMENT,11668.14,41554.0,29885.86,0.0,0.0,0


In [15]:
drop.to_csv('new_fraud2 detection.csv', index=False)